In [1]:
# Notebook Summary:

# V.History: 
# Date Last Modified: 14 May 2025

#This notebook reads 24 years of WQ (EA's open-sourced) data stored in five different datasets (5 years per dataset). 
#Then it combines all five into one dataset to avoid GCP's memory issues when processing large volumes.

#Input File Locations
# 1) gcs://rdmai_data/cleansed/02_WQEA_2000_2004_Raw_New.csv
# 2) gcs://rdmai_data/cleansed/02_WQEA_2005_2009_Raw_New.csv
# 3) gcs://rdmai_data/cleansed/02_WQEA_2010_2014_Raw_New.csv
# 4) gcs://rdmai_data/cleansed/02_WQEA_2015_2019_Raw_New.csv
# 5) gcs://rdmai_data/cleansed/02_WQEA_2020_2024_Raw_New.csv


#Output File Locations
# 1) gcs://rdmai_data/cleansed/02_WQEA_2000_2024_Raw_New.csv #Whole 24 years of WQ data in Raw form as Single dataset
# 2) gcs://rdmai_data/cleansed/03_WQEA_2000_2024_Cleansed_New.csv #Removed redundant columns
# 3) gcs://rdmai_data/cleansed/03_WQEA_2000_2024_Cleansed_Sorted_New.csv #Columns Sorted using -
     #-asceding order on columns: 1. samplingPoint_notation, 2. sampleDateOnly, 3. determinand_notation.
     #A Sequence number added on top of sorted dataset. This is particularly useful Input while modelling.

#Pre-Requisite : 
    #Kernel Python 3 (ipykernel) is required to run this notebook 
    #Required python version - Python 3.10.15 and its compatible Numpy , ScikitLearn libraries

#Old Name: 03_NB_EDA_pre_procs.ipynb

In [2]:
#Check python version compatibility 3.10 or above is required
!python -V
python_version=!(python --version 2>&1)
print (python_version)

Python 3.10.15
['Python 3.10.15']


In [14]:
#Begin CARD
print("Begin Card")

#User-Defined functions

def showtime():
    import time
    
    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print(current_time)
    return()


def savedata(tDF, tname) :
    #tpath = 'gcs://rdmai_data/' #CTS GCP
    tpath = 'gcs://rdmai_dev_data/' #NW GCP
    tclensed = 'cleansed/'
    tDF.to_csv(tpath+tclensed+tname)
    return()


Begin Card


In [4]:
%run "99_NB_CommonUtils.ipynb" #Load Common Utility functions

Intalling required libraries and utilities.....
Uses Python 3 (ipykernel) (Local)
Python 3.10.15
['Python 3.10.15']
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 99.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
06:10:43
welcome
gcs://rdmai_data/raw/
Begin Card



|| Completed intalling required libraries and utilities ||


Mark down 

'''
showtime()

#Library Declaration section 2
#Dataset Maths and OS
#pip install numpy==1.24.3
#pip install seaborn

import pandas as pd
import numpy as np
import os

#Graphs
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", None) 

#Library Declaration section 1
print("welcome")

#Constants declaration for the folder path for the local notebook path
path = 'DataEAOLD/'
raw = 'Raw/'
curated = 'Curated/'
cleansed = 'Cleansed/'

#Constants declaration for the folder path for files stored under Google Cloud Storage 
path = 'gcs://rdmai_data/'
raw = 'raw/'
curated = 'curated/'
cleansed = 'cleansed/'

#Library Declaration section 3
#building the Auto Arima model
#import pmdarima as aa

#Library Declaration section 4
import re
#from datetime_truncate import truncate
from functools import reduce
from datetime import datetime, timezone, timedelta
from datetime import datetime
from datetime import datetime as dt
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#Library Declaration section 5
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV

#Library Declaration section 6
import sklearn
#print(sklearn.show_versions())

#Library Declaration section 7
#pip install imbalanced-learn --user
#pip install imblearn --user
#pip install -U threadpoolctl --user
#import imblearn
#print(imblearn.__version__)

#Library Declaration section 8
# Library Declarations - Model performance matrics 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

#Library Declaration section 9
# Display configurations
pd.set_option('display.max_columns', 400)
#pd.set_option('max_rows', None)

#Read input file
import glob

#import dask.dataframe as dd

import pandas

print (path+raw)
'''

'''
!pip install pyspark | grep -v 'already satisfied'

import pyspark

#Install findspark
!pip install findspark | grep -v 'already satisfied'

# Import findspark
import findspark
findspark.init()

#import pyspark
import pyspark
from pyspark.sql import SparkSession
'''
print()

In [7]:
#Local variable declerations which may be used for naming the columns while read/write
reqcols = 'unique_ID', 'samplingPoint', 'samplingPoint_notation', 'samplingPoint_name', 'sampleDateTime', \
            'determinand_name', 'determinand_definition', 'determinand_notation', 'codedResultInterpretation_interpretation', \
            'determinand_unit_name', 'sampledMaterialType_name', 'purpose_name', 'isComplianceSample', 'result', \
            'samplingPoint_easting', 'samplingPoint_northing'    

In [8]:
#Begin Combine five into one Years 2000 - 2024

In [11]:
''' New logic Security_Indicator:212154 --Pasu
df = pd.DataFrame()
showtime()
df = loaddata()
showtime()
df.head(2)
'''
print()

#NEW Main Read
#Full) Combining 2000 2024

#Constants declaration for the folder path for files stored under Google Cloud Storage 
#path = 'gcs://rdmai_data/' #CTS GCP
path = 'gcs://rdmai_dev_data/' #NW GCP
raw = 'raw/'
curated = 'curated/'
cleansed = 'cleansed/'

all_files = glob.glob(os.path.join(path + cleansed, "*.csv"))
wqpath = 'Water_Quality_EA/'
#wqpath = '' #Make this commented when reading from Google Cloud Storage

print (path+raw+wqpath )

#Read 1st Set from 2000 till 2024
csv_filenames1 = ['2000.csv', '2001.csv', '2002.csv', '2003.csv', '2004.csv']
csv_filenames2 = ['2005.csv', '2006.csv', '2007.csv', '2008.csv', '2009.csv']
csv_filenames3 = ['2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv']
csv_filenames4 = ['2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv']
csv_filenames5 = ['2020.csv', '2021.csv', '2022.csv', '2023.csv', '2024.csv']
#showtime()

df1 = pd.DataFrame()
df1 = pd.read_csv(path+cleansed+'02_WQEA_2000_2004_Raw_New.csv')
print(len(df1))
#showtime()

df2 = pd.DataFrame()
df2 = pd.read_csv(path+cleansed+'02_WQEA_2005_2009_Raw_New.csv')
print(len(df2))
#showtime()

df3 = pd.DataFrame()
df3 = pd.read_csv(path+cleansed+'02_WQEA_2010_2014_Raw_New.csv')
print(len(df3))
#showtime()

df4 = pd.DataFrame()
df4 = pd.read_csv(path+cleansed+'02_WQEA_2015_2019_Raw_New.csv')
print(len(df4))
#showtime()

df5 = pd.DataFrame()
df5 = pd.read_csv(path+cleansed+'02_WQEA_2020_2024_Raw_New.csv')
print(len(df5))
#showtime()


df_2000_2024 = pd.concat((df1, df2, df3, df4, df5), ignore_index=True)
print(len(df_2000_2024))
#Clearing the memory used by the temporary datasets
del(df1, df2, df3, df4, df5)
print ("deleted temp Pandas datasets")



gcs://rdmai_dev_data/raw/Water_Quality_EA/
22210046
17904410
13263201
9411682
5681321
68470660
deleted temp Pandas datasets


In [15]:
#########################################
#Full) Write five years of data at once 2000-2024
showtime()
savedata(df_2000_2024, "02_WQEA_2000_2024_Raw_New.csv")
showtime()
#del(df_2000_2024)
#########################################

06:25:22
06:43:28


()

In [16]:
#End Combine five into one Years 2000 - 2024

In [17]:
#Begin Cleansing

In [18]:
df = df_2000_2024[['sample.samplingPoint.notation', 'sample.samplingPoint.label', 
                   'sample.sampleDateTime', 'determinand.label',
                   'determinand.definition', 'determinand.notation',
                   'result', 'determinand.unit.label', 
                   'sample.sampledMaterialType.label', 'sample.isComplianceSample', 
                   'sample.purpose.label',
                   'sample.samplingPoint.easting','sample.samplingPoint.northing']]
del(df_2000_2024)

In [19]:
df = df.rename(
    columns={"sample.samplingPoint.notation": "samplingPoint_notation", \
             "sample.samplingPoint.label": "samplingPoint_name", \
             "sample.sampleDateTime": "sampleDateTime", \
             "determinand.label": "determinand_name", \
             "determinand.definition": "determinand_definition", \
             "determinand.notation": "determinand_notation", \
             "determinand.unit.label": "determinand_unit_name", \
             "sample.sampledMaterialType.label": "sampledMaterialType_name", \
             "sample.isComplianceSample": "isComplianceSample", \
             "sample.purpose.label": "purpose_name", \
             "result": "result", \
             "sample.samplingPoint.easting": "samplingPoint_easting", \
             "sample.samplingPoint.northing": "samplingPoint_northing" \
            })
df.head(2)

,samplingPoint_notation,samplingPoint_name,sampleDateTime,determinand_name,determinand_definition,determinand_notation,result,determinand_unit_name,sampledMaterialType_name,isComplianceSample,purpose_name,samplingPoint_easting,samplingPoint_northing
0,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04T14:10:00,Ammonia(N),Ammoniacal Nitrogen as N,111,10.40,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020,233400
1,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04T14:10:00,Sld Sus@105C,"Solids, Suspended at 105 C",135,10.00,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020,233400


In [20]:
#Filter the Date out of Datetime
df['sampleDateTime'] = pd.to_datetime(df['sampleDateTime'])
df['sampleDateOnly'] = df['sampleDateTime'].dt.date
#df = df.set_index('sampleDateOnly')
df.head(2)

,samplingPoint_notation,samplingPoint_name,sampleDateTime,determinand_name,determinand_definition,determinand_notation,result,determinand_unit_name,sampledMaterialType_name,isComplianceSample,purpose_name,samplingPoint_easting,samplingPoint_northing,sampleDateOnly
0,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04 14:10:00,Ammonia(N),Ammoniacal Nitrogen as N,111,10.40,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020,233400,2000-01-04
1,AN-011396,BUCKINGHAM GOLF CLUB TINGEWICK RD.,2000-01-04 14:10:00,Sld Sus@105C,"Solids, Suspended at 105 C",135,10.00,mg/l,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),467020,233400,2000-01-04


In [21]:
sorted_df = df.sort_values(["samplingPoint_notation","sampleDateOnly", "determinand_notation"],
                           ascending=[True, True, True])

sorted_df.insert(0, 'sno', range(1, 1 + len(sorted_df)))
sorted_df.set_index('sno', inplace=True)

sorted_df.head(20)


,samplingPoint_notation,samplingPoint_name,sampleDateTime,determinand_name,determinand_definition,determinand_notation,result,determinand_unit_name,sampledMaterialType_name,isComplianceSample,purpose_name,samplingPoint_easting,samplingPoint_northing,sampleDateOnly
sno,,,,,,,,,,,,,,
1,AN-011262,STEANE PARK THE MANOR HOUSE STW,2002-05-28 13:50:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2002-05-28
2,AN-011262,STEANE PARK THE MANOR HOUSE STW,2003-06-11 14:00:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2003-06-11
3,AN-011262,STEANE PARK THE MANOR HOUSE STW,2004-06-28 14:00:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2004-06-28
4,AN-011262,STEANE PARK THE MANOR HOUSE STW,2005-06-03 11:25:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2005-06-03
5,AN-011262,STEANE PARK THE MANOR HOUSE STW,2006-12-13 00:01:00,Oil & Grs Vs,"Visible oil or grease, significant trace: Pres...",664,0.00,pres/nf,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2006-12-13
6,AN-011262,STEANE PARK THE MANOR HOUSE STW,2006-12-13 00:01:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2006-12-13
7,AN-011262,STEANE PARK THE MANOR HOUSE STW,2007-07-05 13:10:00,Oil & Grs Vs,"Visible oil or grease, significant trace: Pres...",664,0.00,pres/nf,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2007-07-05
8,AN-011262,STEANE PARK THE MANOR HOUSE STW,2007-07-05 13:10:00,D Site Insp,Descriptive Site Inspection : Pass/Fail 1/0,7444,1.00,coded,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2007-07-05
9,AN-011262,STEANE PARK THE MANOR HOUSE STW,2008-02-15 10:35:00,Oil & Grs Vs,"Visible oil or grease, significant trace: Pres...",664,0.00,pres/nf,FINAL SEWAGE EFFLUENT,True,COMPLIANCE AUDIT (PERMIT),455510,239070,2008-02-15


In [22]:
print("Total Volume: ", len(df), len(sorted_df))

68470660 68470660


In [23]:
#########################################
#Save Combined and cleaned data
showtime()
savedata(df, "03_WQEA_2000_2024_Cleansed_New.csv")
showtime()
del(df)

#Save Combined and cleaned data
showtime()
savedata(sorted_df, "03_WQEA_2000_2024_Cleansed_Sorted_New.csv")
showtime()
del(sorted_df)
#########################################

06:44:45
06:58:12
06:58:12
07:11:36


In [24]:
#End Cleansing

In [25]:
#END CARD
#In line comments completed 09-May-2025